## selenium을 이용하여 주유가격 수집
- 매일 1회 수집
- 지역별 검색 사이트에 진입
- 시/도 => wait => 시/군/구 => wait => 읍/면/동 변경됨
- 조회 버튼 클릭
- 엑셀저장 클릭
- 이렇게 돌면서 다 파일을 모아두고 *.xls을 한번에 목록을 읽어드리고 (전처리가 잘되어있는 파일들이라서 덩어리만 잘가지고오면된다)
- pandas의 엑셀을 DataFrame으로 로드하는 함수를 활용하여 변환후
- df의 머리를 이용하여 덩어리로 만들어서 디비에 삽입 or 매번 삽입


In [48]:
from selenium import webdriver as wd
import urllib

In [49]:
driver = wd.Chrome('../tool/chromedriver75.0.3770.140.exe')

In [50]:
target_url = 'http://www.opinet.co.kr/searRgSelect.do'

In [51]:
driver.get( target_url)
import time
time.sleep(5)

In [52]:
driver.get( target_url) # 서버사이드 못들어가게 막는다

In [53]:
# 테스트 사이트에서 검사 -> 서울 찾기 -> select id(SIDO_NM0)를 찾아라

In [54]:
# 시도 정보 추출 및 특정 시도선택
# select tag이고, 이 요소의 값을 설정하면 시도값이 변경된다
sido = driver.find_element_by_id('SIDO_NM0')
# 그 밑의 옵션 태그들

In [55]:
# 시도를 선택하는 작업
# sido.send_keys('서울특별시') # 크롬브라우저로 띄운것에는 처음에는 부산으로 잡혀있음 -> 서울특별시로 변경됨

In [56]:
# 시도에 들어올 수 있는 값 추출 (서울특별시로 바뀌는 근거를 찾아보는 것)
# select > option들의 속성중에 value값
# #SIDO_NMO > 직계 자식중의 option
# for option in driver.find_elements_by_css_selector('#SIDO_NM0 > option'): # sido 밑에 있는 그 자손값인 option을 전부 다겨와라(리스트됨)
#    print( option.get_attribute('value') )
sidos = [ option.get_attribute('value') 
        for option in driver.find_elements_by_css_selector('#SIDO_NM0 > option')
        if option.get_attribute('value')
        ]
sidos

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [57]:
for sido_name in sidos:
    sido.send_keys(sido_name)
    time.sleep(2)    # 조회찾기의 지역값들이 2초마다 바뀐다 -> 이것을 3번 쳐야한다 -> 지역, 시/군, 시/군/구

In [66]:
sido = driver.find_element_by_id('SIDO_NM0')
sido.send_keys('부산')

In [67]:
# 시군구 선택
# 사이트 특성상 send_keys()를 수행하면 화면이 전부 리로드된다(화면이 껌벅인다)
# 그래서 이전에 잡아두었던 sigungo값들이 전부 무효화 한다
# 해서 다시 sigungo변수를 찾아서 처리해야한다
sigungu = driver.find_element_by_id('SIGUNGU_NM0')
sigungu.send_keys('남구')

In [68]:
# 읍면동 선택
dong = driver.find_element_by_id('DONG_NM')
dong.send_keys('대연동')

In [69]:
# 조회버튼 클릭
driver.find_element_by_id('searRgSelect').click()

In [70]:
# 엑셀저장 클릭: 버튼 누르는 것 (ex) 엑셀저장버튼 누르기
driver.find_element_by_id('glopopd_excel').click()

In [71]:
driver.close()
driver.quit()